# 기본 Linear Regression

In [1]:
import numpy as np
import torch

# Feed Forward (순전파)
* b = -1, w = 2
* y_hat = 2x + (-1)

In [2]:
# 임의의 w,b를 초기에 설정
w = torch.tensor(2.0)
b = torch.tensor(-1.0)

print(w)
print(b)

tensor(2.)
tensor(-1.)


In [3]:
x = torch.tensor([1.0])
x

tensor([1.])

In [4]:
def forward(x):
    yhat = w * x + b
    return yhat

In [5]:
yhat = forward(x)
yhat

tensor([1.])

* 다항 회귀 (x가 여러개)

In [7]:
x = torch.tensor([[1.0], [2.0], [3.0]])
forward(x)

tensor([[1.],
        [3.],
        [5.]])

In [8]:
# numpy 객체로 실행
x = np.arange(1,4).reshape(3,1)
x

array([[1],
       [2],
       [3]])

In [9]:
forward(x)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22756\1072465637.py:2: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  yhat = w * x + b


tensor([[1.],
        [3.],
        [5.]], dtype=torch.float64)

## 2. torch.nn의 Linear 클래스를 이용한 Linear Regression
* b = 0.44, w = 0.5135
* y_hat = 0.44 + 0.5135 * x

In [11]:
from torch.nn import Linear
import torch
torch.manual_seed(1)

# 선형 회귀 모델 생성
# 입력 특성 수 1, 출력 특성 수 1
# bias = True : 모델에 편향(bias) 값을 포함
model = Linear(in_features=1, out_features=1, bias=True)
model

Linear(in_features=1, out_features=1, bias=True)

In [12]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


In [13]:
print(model.state_dict())
print()
print(model.state_dict()['weight'], model.state_dict()['bias'])
print()
print(model.weight)
print(model.bias)

OrderedDict([('weight', tensor([[0.5153]])), ('bias', tensor([-0.4414]))])

tensor([[0.5153]]) tensor([-0.4414])

Parameter containing:
tensor([[0.5153]], requires_grad=True)
Parameter containing:
tensor([-0.4414], requires_grad=True)


In [14]:
# weight와  bias의 값 update
model.state_dict()['weight'][0] = torch.tensor(0.5135)
model.state_dict()['bias'][0] = torch.tensor(-0.44)

In [15]:
model.state_dict()

OrderedDict([('weight', tensor([[0.5135]])), ('bias', tensor([-0.4400]))])

In [16]:
# x = [[0.0]]일 경우 prediction --> 0.0 * 0.5153 - (0.44) = 0.734
x = torch.tensor([[1.0]])
yhat = model(x)
print(yhat.detach().numpy())

[[0.07349998]]


* w = 2, b=-1 경우, yhat의 결과는?

In [17]:
# weight와  bias의 값 update
model.state_dict()['weight'][0] = torch.tensor(2)
model.state_dict()['bias'][0] = torch.tensor(-1)
model.state_dict()

OrderedDict([('weight', tensor([[2.]])), ('bias', tensor([-1.]))])

In [18]:
# x = [[1.0]]일 경우 prediction --> 1.0 * 2 +(-1) = 1
x = torch.tensor([[1.0]])
yhat = model(x)
yhat.item()

1.0

In [19]:
# x 가 multiple inputs (array) 인 경우:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
print(yhat.detach().numpy())

[[1.]
 [3.]
 [5.]]


## 3. 사용자 정의 Module 작성
### 학습 클래스, 순전파 정의

In [23]:
from torch import nn

class LR(nn.Module):
    def __init__(self, input_size, output_size):
        # super(LR, self).__init__() # python 2.x 문법
        super().__init__()      # python 3.x 문법
        # 선형 레이어를 정의합니다. 입력 크기와 출력 크기를 인수로 받습니다.
        self.linear = nn.Linear(input_size, output_size)

    # 순전파 정의: 모델 학습을 
    def forward(self, x):
        # 입력 x를 선형 레이어에 통과시켜 출력값을 계산합니다.
        out = self.linear(x)
        return out # 최종 출력값 반환

### 모델 객체 생성

In [24]:
# 입력 size 1, 출력 size 1
model = LR(1, 1)
model.state_dict()

OrderedDict([('linear.weight', tensor([[-0.1939]])),
             ('linear.bias', tensor([0.4694]))])

### weight, bias 업데이트

In [25]:
# 2) 번의 실습 내용과 같은지 확인하기 위해
# weight, bias 설정
model.state_dict()['linear.weight'][0] = torch.tensor(2)
model.state_dict()['linear.bias'][0] = torch.tensor(-1)

print(list(model.parameters()))
print()
print(model.linear)

[Parameter containing:
tensor([[2.]], requires_grad=True), Parameter containing:
tensor([-1.], requires_grad=True)]

Linear(in_features=1, out_features=1, bias=True)


### 테스트 하기
* 예측 계산

In [27]:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)

# 계산 그래프로부터 텐서를 분리하여 그라디언트 계산이 필요 없는 텐서로 만듭니다
print(yhat)
# print(yhat.detach())  # tracking을 끊어주는 역할
# print(yhat.detach().numpy())

tensor([[1.],
        [3.],
        [5.]], grad_fn=<AddmmBackward0>)


### 모델 매개변수 확인

In [28]:
list(model.parameters())

[Parameter containing:
 tensor([[2.]], requires_grad=True),
 Parameter containing:
 tensor([-1.], requires_grad=True)]